## 0 Intro

本 note 关注于实现神经网络中常用的层，设计原则如下：

1. 每一层均使用一个类来实现
2. 每一个类维护前向计算时所需要的**参数**、后向计算时所需要的**中间结果**、更新参数时所需要的**梯度**
3. 每个类的**参数**和**梯度**用列表 `params` 和 `grads` 打包，便于整个神经网络的训练
4. 每一个类至少包含三个方法，即 `__init__()`、`forward()`、`backward()`
5. 复杂类可以基于简单类来实现

**备注**
> 梯度的形状和对应参数的形状一致

## 1 MatMul Layer

本层执行矩阵乘法运算，即 $\mathbf{y} = \mathbf{x}\mathbf{W}$，这里不考虑偏置值

实现如下：

**成员变量**
- 参数： `W` - 参数矩阵
- 中间结果 `x` - 输入数据
- 梯度: `dW`

**成员函数**

- `__init__(self, W)`
- `forward(self, x)`
- `backward(self, dout)`

代码如下：

In [ ]:
class MatMul:
    def __init__(self, W):
        '''初始化所需要维护的成员变量，其中参数 W 由外界传入
        '''
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.x = None
        
    def forward(self, x):
        '''前向计算
        
        :param x: 是输入数据
        
        :return: 输出矩阵乘法的结果
        '''
        W, = self.params
        self.x = x
        out = np.dot(x, W)
        
        return out
    
    def backward(self, dout):
        '''后向计算，计算更新本层参数的梯度，以及传播到后一层的导数
        
        :param dout: 上游来的导数信号
        
        :return dx: 传给下游的导数信号
        '''
        ## 这里考虑了 x 是批数据的情况
        W, = self.params
        dW = np.dot(self.x.T, dout)
        dx = np.dot(dout, W.T)
        
        self.grads[0][...] = dW  # 注意这里的深拷贝
        
        return dx

## 2 Affine Layer

Affine 层实现的仿射变换，即 $\mathbf{y}=\mathbf{x}\mathbf{W} + \mathbf{b}$，相比 MatMul，Affine 多了一个偏置值的计算

实现如下：

**成员变量**

- 参数：`W` - 矩阵； `b` - 偏置值;
- 中间结果: `x` - 输入数据
- 梯度: `dW`；`db`

**成员函数**

- `__init__(self, W, b)`
- `forward(self, x)`
- `backward(self, dout)`

代码如下：

In [ ]:
class Affine:
    def __init__(self, W, b):
        '''初始化 Affine 所维护的参数，参数 W 和 b 由外界传入
        '''
        self.params = [W, b]  # 参数列表
        self.grads = [np.zeros_like(W), np.zeros_like(b)]  # 梯度列表
        self.x = None
        
    def forward(self, x):
        '''前向计算
        '''
        W, b = self.params
        self.x = x
        out = np.dot(x, W) + b
        
        return out
    
    def backward(self, dout):
        '''后向计算，计算更新本层参数所需的梯度，以及下游所需的导数信号
        '''
        W, b = self.params
        
        dW = np.dot(self.x.T, dout)
        db = np.sum(dout, axis=0)
        dx = np.dot(dout, W.T)
        
        self.grads[0][...] = dW
        self.grads[1][...] = db
        
        return dx

## 3 Softmax Layer

softmax 层完成的是如下的计算 $\mathbf{y} = softmax(\mathbf{x})$

不失一般性，这里仅考虑 $\mathbf{x}$ 是一维向量或者二维批数据的情况

**成员变量** （准确讲，softmax 没有需要学习的参数，为了可以统一处理，使用空 `params` 和 `grads` 来替代）

- `out` - 输出结果

**成员函数**

- `__init__(self)`
- `forward(self, x)`
- `backward(self, dout)`

实现如下：

In [ ]:
class Softmax:
    def __init__(self):
        self.params, self.grads = [], []
        self.out = None
        
    def forward(self, x):
        self.out = softmax(x)
        return self.out
    
    def backward(self, dout):
        

## 4 SoftmaxWithLoss Layer

本层可以看做是 Softmax 层后接一个交叉熵计算所形成的复合层，softmax + 交叉熵是比较常见的用于多分类问题的组合

实现如下：

**成员变量**

- 没有需要学习的参数，故 `params` 和 `grads` 均是空列表
- 中间结果 `y` - softmax 的输出；`t` - 训练数据中的标签
- 梯度 -  `grads` 为空列表

**成员函数**

- `__init__(self)`
- `forward(self, x)`
- `backward(self, dout=1)` - 因为一般作为神经网络的输出，所以 `dout=1`

实现如下：

In [1]:
class SoftmaxWithLoss:
    def __init__(self):
        self.params, self.grads = [], []  # 没有需要学习的参数，用空列表代替便于处理的一致性
        self.y = None  # softmax 的输出
        self.t = None  # 训练数据中的标签
        
    def forward(self, x, t):
        self.y = softmax(x)
        self.t = t
        
        ## 如果 t 是 one-hot 标签，转换为正确标签的索引值
        if self.t.size == self.y.size:
            self.t = self.t.argmax(axis=1)
        
        loss = cross_entropy_error(self.y, self.t)
        
        return loss
    
    def backward(self, dout=1):
        '''后向传播计算，因为本层一般是作为模型的输出层，所以上游来的导数默认为1
        '''
        ## 默认作为 mini-batch 来处理
        batch_size = self.t.shape[0]
        
        ## 在推导 dx = dout * (y-t)，默认 t 是以 one-hot 的格式出现
        ## 所以在计算 y-t 时，可以只计算正确标签对应位置的减法，因为其他位置 t 元素值为 0
        ## 即只要拿到正确标签的索引值即可
        ## 这里需要先处理 one-hot 格式的 t，以获取正确标签的索引值
        
        if self.t.size == self.y.size:
            self.t = self.t.argmax(axis=1)
            
        dx = self.y.copy()  # 深拷贝一份 y
        dx[np.arange(batch_size), self.t] -= 1  # 计算 y-t 的技巧
        dx = dx * dout
        dx = dx / batch_size  # 共 batch-size 个数据参与计算交叉熵，每条数据贡献需要除以数据量
        
        return dx

## 5 Sigmoid Layer

sigmoid 层完成的计算如下 $\mathbf{y} = \frac{1}{1 + \exp(-\mathbf{x})}$

sigmoid 一般用于完成二分类问题，实现如下：

**成员变量**

- 参数 - 没有需要学习的参数，故 `params`是空列表
- 中间变量 - `out` 前向计算的输出 out
- 梯度 -  `grads` 为空列表

**成员函数**

- `__init__(self)`
- `forward(self, x)`
- `backward(self, dout)`

代码如下：

In [2]:
class Sigmoid:
    def __init__(self):
        self.params, self.grads = [], []
        self.out = None
        
    def forward(self, x):
        out = 1 / 1 + np.exp(-x)
        self.out = out
        
        return out
    
    def backward(self, dout):
        dx = dout * self.out * (1 - self.out)
        
        return dx